# Procesar los datos de los videos

Este notebook se encarga de transformar los datos crudos provenientes de archivos de video en un formato estructurado y útil para el análisis. Parte de archivos que almacenan la posicion tridimensional de distintos puntos del cuerpo, después calcula el ángulo entre disintos puntos del cuepro, y por último se realizan cálculos estadísticos sobre los ángulos

**Objetivo**
*  Leer y procesar los archivos crudos para crear un DataFrame que contenga información esencial sobre cada grabación, incluyendo detalles sobre el paciente, el número de repetición, el ID del movimiento, la precisión del gesto (si es correcto o no), y la posición de los keypoints.
* Crear un segundo DataFrame que contenga los ángulos calculados entre algunos de los keypoints, lo que ayudará a entender mejor la postura y el movimiento durante la ejecución del gesto.
* Sobre este último, realizar cálculos estadísticos sobre los ángulos calculados entre keypoints para cada repetición y gesto. 

**Entrada**
* Archivos de datos de videos dentro del directorio ``SkeletonData/RawData``, que contienen información sobre el paciente, las repeticiones, los IDs de movimiento, y las posiciones de keypoints.

**Salida**
* ``Resultados/raw_pacientes.csv``: contiene información detallada sobre cada grabación. Es el archivo de entrada de ``analisis_datos.ipynb``.
* ``Resulados/angles.csv``: incluye los ángulos calculados entre keypoints, facilitando el análisis de la postura y el movimiento durante el gesto. Este será el archivo de entrada de ``medidas_angulos.ipynb`` y de ``analisis_datos.ipynb``.
* ``Resultados/medidasPerRepetition.csv``: contiene una fila por repetición y gesto, que incluye estadísticas para cada ángulo calculado.

**Índice**
1. [Procesar Raw Data Folder](#1-procesar-raw-data-folder)\
    1.1 [Datos de los nombres de los archivos](#11-datos-de-los-nombres-de-los-archivos)\
    1.2 [Datos de los archivos](#12-datos-de-los-archivos)
2. [Calcular los ángulos](#2-calcular-los-ángulos)
3. [Cálculos estadísticos sobre los ángulos](#3-cálculos-estadísticos-sobre-los-ángulos)


In [1]:
# importar librerias necesarias
import pandas as pd # para manejar dataframes
import functions as fn
import os

# Para ignorar los FutureWarning
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

*****
## 1. Procesar Raw Data Folder

El conjunto de datos incluye a 30 sujetos, de los cuales 14 son sanos y 16 son pacientes. Cada participante realiza 9 tipos de ejercicios, con 6 repeticiones de cada uno.

La carpeta contiene 2598 archivos de texto con información de los videos, donde cada archivo corresponde a un sujeto realizando una repetición de un gesto. Hay información tanto en el contenido del archivo como en el propio nombre del archivo.

* En el nombre del archivo se encuentran los identificadores del sujeto, del día, del gesto, el número de repetición, si es correcto y la posición del sujeto durante el ejercicio.

* Dentro de cada archivo hay información sobre las coordenadas de 25 puntos del cuerpo captados por un sensor que graba a 30 fps. Por lo tanto, por cada segundo de video, se generan 30 líneas en el archivo de texto. Cada línea sigue el siguiente formato: marca de tiempo, XX, XX, seguido de 25 pares de (NombreArticulación, EstadoSeguimiento, coordenada 3D X, coordenada 3D Y, coordenada 3D Z, coordenada 2D X, coordenada 2D Y).


### 1.1 Datos de los nombres de los archivos

Los nombre de los archivos siguen el siguiente formato: *SubjectID_DateID_GestureLabel_RepetitionNumber_CorrectLabel_Position.txt*

* *SubjectID*: id uniquely identifying the person performing the exercise
* *DateID*: id identifying the session in which the person was recorded
* *GestureLabel*: Label identifying the gesture; possible values are from 0 to 8
* *RepetitionNumber*: Each gesture was repeated several times and this shows the repetition number
* *CorrectLabel*: A value of 1 represents a gesture labeled as being correctly executed, while a value of 2 is for a gesture labeled as incorrect
* *Position*: Some of the persons performed the gestures sitting on a chair or wheelchair, while others standing

### 1.2 Datos de los archivos
Each raw data file contains per line: timestamp, XX, XX, followed by a 25 pairs of (JointName, TrackedStatus, 3d coordinate X, 3d coordinate Y, 3d coordinate Z, 2d coordinate X, 2d coordinate Y)

In [2]:
# Función para extraer la información de dentro de los archivos
def leer_datos_archivo(directorio:str, columnas:list[str]) -> pd.DataFrame:
    """
    Compila la información de los archivos en un directorio y los guarda en un DataFrame.

    Parámetros
    ----------
    directorio : str
        Nombre del directorio donde se encuentran los archivos.
    columnas : list[str]
        Lista con los nombres de las columnas para el DataFrame de salida.

    Return
    -------
    pd.DataFrame
        DataFrame con todos los datos recopilados de los archivos.
    """
    # Crea una lista con los nombres de los archivos en el directorio
    file_list = os.listdir(directorio)

    # lista para almacenar los datos extraídos
    list_data = []

    # Itera sobre cada archivo
    for file_name in file_list:
         # Extrae los datos del nombre del archivo
        campos = fn.leer_nombre_archivo(file_name)

        with open(os.path.join(directorio, file_name), 'r') as file:
            for line in file:
                 # Divide la línea por comas y extrae la información deseada
                 # omitiendo timestamp y otros datos innecesarios
                line_data = line.strip().split(',')[3:]
                # Quita los paréntesis
                cleaned_data = [item.replace('(', '').replace(')', '') for item in line_data] 
                # por cada linea de los archivos necesitamos bloques de 7 valores
                for i in range(0, len(cleaned_data), 7): 
                    list_data.append(campos + cleaned_data[i:i + 7])
    df = pd.DataFrame(list_data, columns=columnas)
    return df

In [3]:
# directorio donde se encuentran los datos
directory = '../dataset/SkeletonData/RawData' 

columnas = ['SubjectID', 'DateID', 'GestureLabel', 'RepetitionNumber', 'CorrectLabel', 'Position',
            'JointName', 'TrackedStatus', '3D_X', '3D_Y', '3D_Z', '2D_X', '2D_Y']

# Extraer la informacion y almacenarla en un DataFrame
df_data = leer_datos_archivo(directory, columnas)

In [4]:
# Eliminar columnas innceserarias
df_data.drop(['TrackedStatus', 'DateID', 'Position', '2D_X', '2D_Y'], axis=1, inplace=True)

# Añadir columna más descriptiva de los gestos
gesture_mapping = {
    '0': 'EFL',
    '1': 'EFR',
    '2': 'SFL',
    '3': 'SFR',
    '4': 'SAL',
    '5': 'SAR',
    '6': 'SFE',
    '7': 'STL',
    '8': 'STR'
}

# Nueva columna basada en el mapeo
columna_gestrure_name = df_data['GestureLabel'].map(gesture_mapping)

# Insertar la columna al lado de GestureLabel
df_data.insert(2, 'GestureName', columna_gestrure_name)

df_data

,SubjectID,GestureLabel,GestureName,RepetitionNumber,CorrectLabel,JointName,3D_X,3D_Y,3D_Z
0,101,0,EFL,1,1,SpineBase,-0.1028086,0.06965441,2.464606
1,101,0,EFL,1,1,SpineMid,-0.1026228,0.3837799,2.438919
2,101,0,EFL,1,1,Neck,-0.1025293,0.6877351,2.40196
3,101,0,EFL,1,1,Head,-0.1190992,0.8358598,2.373549
4,101,0,EFL,1,1,ShoulderLeft,-0.2826451,0.5525576,2.38421
...,...,...,...,...,...,...,...,...,...
5206170,307,8,STR,9,1,SpineShoulder,-0.05799517,0.5291457,2.422904
5206171,307,8,STR,9,1,HandTipLeft,-0.302538,-0.1131345,2.284269
5206172,307,8,STR,9,1,ThumbLeft,-0.2783904,-0.06298634,2.269769
5206173,307,8,STR,9,1,HandTipRight,0.1525867,-0.136378,2.45287


In [5]:
# guardar los datos en formato csv
df_data.to_csv('../Resultados/raw_pacientes.csv', index=False)

******
## 2. Calcular los ángulos

<div style="text-align: center;">
<img src="../Imagenes/gestures.png" width="500"/>
</div>

In [6]:
# Aplica la función para caluclar los ángulos a los datos en crudo
df_angles = fn.apply_angles_I(df_data)
# Visualizar el dataframe
df_angles

,SubjectID,GestureLabel,GestureName,RepetitionNumber,CorrectLabel,ElbowAngleLeft,ElbowAngleRight,ShoulderAngleLeft,ShoulderAngleRight,WristAngleLeft,WristAngleRight,HipAngleLeft,KneeAngleLeft,AnkleAngleLeft,HipAngleRight,KneeAngleRight,AnkleAngleRight,LeftArmAngle,RightArmAngle,ArmsTogetherAngle
0,101,0,EFL,1,1,18.731844,12.815119,142.560727,140.857143,2.598764,6.344406,104.574768,3.743551,64.246261,101.521850,4.855627,60.940986,156.791112,159.877408,152.580270
1,101,0,EFL,1,1,18.682011,13.214648,142.757760,140.967721,2.522847,5.905771,104.499195,3.821802,64.334729,101.590425,4.885456,61.134582,156.667813,160.035203,152.602527
2,101,0,EFL,1,1,18.530184,14.175895,142.724795,140.886094,3.239981,6.639703,104.558119,3.832570,64.078901,101.492094,4.776421,59.972082,156.747131,160.251764,152.596513
3,101,0,EFL,1,1,18.525441,15.675640,142.796223,140.996819,3.488477,10.172814,104.502499,3.642046,63.451166,101.422947,4.655747,59.366231,156.715615,160.589502,152.653362
4,101,0,EFL,1,1,18.440644,18.672491,142.755019,140.986030,3.640028,9.700975,104.554046,3.670732,63.212740,101.311001,4.601472,59.328043,156.793484,160.668306,152.675640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208242,307,8,STR,9,1,19.235168,16.366695,142.466122,142.949282,10.597409,8.590306,107.710008,17.152761,57.808079,94.948037,1.709497,41.616584,156.103380,158.651430,149.864559
208243,307,8,STR,9,1,19.337504,16.460773,142.614506,142.959204,9.097187,8.083043,107.352914,16.479229,57.733793,94.653231,1.263963,41.879291,156.056261,158.714378,149.734213
208244,307,8,STR,9,1,19.762179,16.292120,142.579738,142.903745,9.288112,10.157666,107.398951,16.368690,58.670153,94.346427,1.041109,78.704138,156.130164,158.836560,149.712017
208245,307,8,STR,9,1,20.339779,16.195526,142.685324,142.897308,7.335038,9.108455,107.436579,16.163032,57.798046,94.144821,1.015447,78.752737,156.114139,158.911807,149.496620


In [7]:
# guardar los datos en formato csv
df_angles.to_csv('../Resultados/angles.csv', index=False)

******
## 3. Cálculos estadísticos sobre los ángulos

Realizar cálculos estadísticos sobre los ángulos calculados entre keypoints para cada repetición y gesto. 

In [8]:
# Cálculos estadísticos sobre los ángulos
def calculos_estadísticos(df:pd.DataFrame) -> pd.DataFrame:
    """
    Realiza cálculos estadísticos sobre los ángulos en un
    DataFrame agrupado por sujeto, gesto y número de repetición.

    Parámetros
    ----------
    df : pd.DataFrame
        DataFrame que contiene información sobre los ángulos,
        así como otras columnas relacionadas con el sujeto,
        gesto, repetición, etc.

    Returns
    -------
    pd.DataFrame
        DataFrame que contiene las estadísticas descriptivas
        calculadas para cada grupo de ángulos, con una fila por
        combinación de sujeto, gesto y repetición.
    """

    # Agrupa el DataFrame 
    groups = df.groupby(["SubjectID", "GestureLabel", "RepetitionNumber"])

    # Lista para almacenar los datos de salida
    data = []

    # Itera sobre cada grupo
    for (subject_id, gesture_label, repetition_number), group in groups:
        # Selecciona solo las columnas que contienen los ángulos 
        angles = group.iloc[:, 6:]

        # Calcula la media y la desviación estándar para los ángulos
        means = angles.mean()
        std_devs = angles.std()

        # Almacena las estadísticas en un diccionario 
        data.append({
            'SubjectID': subject_id,
            'GestureLabel': gesture_label,
            'GestureName': group['GestureName'].iloc[0],
            'RepetitionNumber': repetition_number,
            'CorrectLabel': group['CorrectLabel'].iloc[0],
            'Duration': len(group),  # Duración en número de frames
            'standardDeviation': std_devs,
            'Maximum': angles.max(),
            'Minimum': angles.min(),
            'Mean': means,
            'Range': angles.max() - angles.min(),
            'Variance': angles.var(),
            'CoV': std_devs / means,  # Coeficiente de variación
            'Skewness': angles.skew(),  # Asimetría
            'Kurtosis': angles.kurtosis()  # Curtosis
        })

    # Convierte la lista de diccionarios en un DataFrame y lo ordena
    df_stats = pd.DataFrame(data)
    df_stats = df_stats.sort_values(['SubjectID', 'GestureLabel', 'RepetitionNumber'])

    return df_stats

In [9]:
# Cálculos sobre los ángulos
df_stats = calculos_estadísticos(df_angles)
df_stats.head()

,SubjectID,GestureLabel,GestureName,RepetitionNumber,CorrectLabel,Duration,standardDeviation,Maximum,Minimum,Mean,Range,Variance,CoV,Skewness,Kurtosis
0,101,0,EFL,1,1,59,ElbowAngleRight 39.820354 ShoulderAngleL...,ElbowAngleRight 120.694659 ShoulderAngle...,ElbowAngleRight 12.815119 ShoulderAngle...,ElbowAngleRight 77.945100 ShoulderAngle...,ElbowAngleRight 107.879540 ShoulderAngle...,ElbowAngleRight 1585.660629 ShoulderAngl...,ElbowAngleRight 0.510877 ShoulderAngleLe...,ElbowAngleRight -0.474274 ShoulderAngleLe...,ElbowAngleRight -1.509605 ShoulderAngleLe...
1,101,0,EFL,2,1,44,ElbowAngleRight 37.944397 ShoulderAngleL...,ElbowAngleRight 130.064917 ShoulderAngle...,ElbowAngleRight 18.847589 ShoulderAngle...,ElbowAngleRight 83.094436 ShoulderAngle...,ElbowAngleRight 111.217328 ShoulderAngle...,ElbowAngleRight 1439.777236 ShoulderAngl...,ElbowAngleRight 0.456642 ShoulderAngleLe...,ElbowAngleRight -0.422067 ShoulderAngleLe...,ElbowAngleRight -1.280617 ShoulderAngleLe...
2,101,0,EFL,3,1,53,ElbowAngleRight 49.054706 ShoulderAngleL...,ElbowAngleRight 160.259284 ShoulderAngle...,ElbowAngleRight 17.456790 ShoulderAngle...,ElbowAngleRight 99.344348 ShoulderAngle...,ElbowAngleRight 142.802494 ShoulderAngle...,ElbowAngleRight 2406.364165 ShoulderAngl...,ElbowAngleRight 0.493785 ShoulderAngleLe...,ElbowAngleRight -0.288228 ShoulderAngleLe...,ElbowAngleRight -1.455247 ShoulderAngleLe...
3,101,0,EFL,4,1,57,ElbowAngleRight 48.916369 ShoulderAngleL...,ElbowAngleRight 153.020930 ShoulderAngle...,ElbowAngleRight 16.267619 ShoulderAngle...,ElbowAngleRight 93.637581 ShoulderAngle...,ElbowAngleRight 136.753311 ShoulderAngle...,ElbowAngleRight 2392.811130 ShoulderAngl...,ElbowAngleRight 0.522401 ShoulderAngleLe...,ElbowAngleRight -0.272649 ShoulderAngleLe...,ElbowAngleRight -1.514596 ShoulderAngleLe...
4,101,0,EFL,5,1,60,ElbowAngleRight 47.890590 ShoulderAngleL...,ElbowAngleRight 154.251172 ShoulderAngle...,ElbowAngleRight 12.984685 ShoulderAngle...,ElbowAngleRight 99.017306 ShoulderAngle...,ElbowAngleRight 141.266487 ShoulderAngle...,ElbowAngleRight 2293.508570 ShoulderAngl...,ElbowAngleRight 0.483659 ShoulderAngleLe...,ElbowAngleRight -0.442398 ShoulderAngleLe...,ElbowAngleRight -1.360146 ShoulderAngleLe...


Como se ve en la cabecera del DataFrame anterior es poco legible, por lo que se va a formatear para mejorar la legibilidad del DataFrame y evitar la representación diccionarios dentro de las columnas del DataFrame

In [10]:
columnas = ['standardDeviation', 'Maximum', 'Minimum', 'Mean', 'Range',
            'Variance', 'CoV', 'Skewness', 'Kurtosis']
# Reformatea cada columna de diccionario y concatena los resultados
nuevas_columnas = pd.concat([fn.formatear_columnas(df_stats[col], col) for col in columnas], axis=1)

# Concatenar las nuevas columnas con el DataFrame original
df = pd.concat([df_stats, nuevas_columnas], axis=1)

# Elimina las columnas originales que contenían diccionarios
df = df.drop(columnas, axis=1)

# Ordena el DataFrame 
## pasamos a numerico para que ordene bien los numeros
df = df.apply(pd.to_numeric, errors='ignore')
df = df.sort_values(['SubjectID', 'GestureLabel', 'RepetitionNumber'])

# Mostrar el resultado
df

,SubjectID,GestureLabel,GestureName,RepetitionNumber,CorrectLabel,Duration,ElbowAngleRight_standardDeviation,RightArmAngle_standardDeviation,ShoulderAngleLeft_standardDeviation,AnkleAngleLeft_standardDeviation,...,ArmsTogetherAngle_Kurtosis,KneeAngleRight_Kurtosis,LeftArmAngle_Kurtosis,HipAngleLeft_Kurtosis,HipAngleRight_Kurtosis,WristAngleLeft_Kurtosis,AnkleAngleRight_Kurtosis,KneeAngleLeft_Kurtosis,WristAngleRight_Kurtosis,ShoulderAngleRight_Kurtosis
0,101,0,EFL,1,1,59,39.820354,2.765557,0.160506,0.658147,...,-0.604303,-0.628212,0.537440,0.062256,-0.924047,-1.154852,-0.404407,1.771001,0.198533,0.247256
1,101,0,EFL,2,1,44,37.944397,2.298280,0.140486,0.831351,...,0.893421,-0.581136,0.726806,-1.078167,-0.198608,-0.770170,-0.276498,-0.363962,1.048145,-1.751783
2,101,0,EFL,3,1,53,49.054706,3.445304,0.378726,0.960267,...,-0.413869,-0.166022,0.140506,-0.941266,-1.007602,-0.580654,-0.530532,0.195184,-0.393123,0.719889
3,101,0,EFL,4,1,57,48.916369,3.626022,0.365630,0.573168,...,-1.258808,-1.467114,1.060234,-1.440586,-1.487544,-0.136403,-0.057357,-0.688205,-0.610934,0.495865
4,101,0,EFL,5,1,60,47.890590,3.767307,0.447574,0.658998,...,-0.338747,-0.170622,-1.527706,-1.091646,-1.427344,-0.426884,-0.322273,1.957381,-1.170145,0.393907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2447,307,8,STR,8,1,48,0.502135,1.555034,0.589366,10.353342,...,-0.339300,-0.038718,-0.781945,-1.222994,-1.014746,-0.949755,-0.270964,-0.347162,0.056064,0.465005
2448,307,8,STR,9,1,48,0.630995,1.332626,0.367367,4.728600,...,-0.863515,-1.195026,-1.644095,-1.672822,-1.651462,0.520658,-1.671159,-0.874793,-1.006323,-0.019000
2438,307,8,STR,10,1,47,0.485371,1.315829,0.546749,8.033854,...,-0.449516,-0.471100,-0.826005,-1.403915,-1.103517,-0.834273,-2.061997,-1.055977,-0.776532,-1.203659
2439,307,8,STR,11,1,51,0.578282,1.896625,0.586437,9.185235,...,-0.445218,-0.274878,-0.620023,-1.462976,-1.697380,-0.261026,-1.011064,-1.100134,-0.052683,-0.087088


In [11]:
# Guardar el DataFrame en formato csv
df.to_csv('../Resultados/medidasPerRepetition.csv', index=False)